In [1]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops, stops
import os
import pickle
from opencc import OpenCC
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer
from transformers import BertTokenizer , BertConfig , BertModel , XLNetTokenizer, XLNetConfig , XLNetModel

import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable

from torch import nn


In [2]:
# train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./tbrain/tbrain_train.csv")

train_df = train_df.fillna('None')

# test_df = pd.read_csv("./tbrain/tbrain_test_split.csv")
test_df = pd.read_csv("./tbrain/tbrain_test.csv")

train_df = pd.concat([train_df,test_df])

test_df = test_df.fillna('None')

print(train_df.shape)
print(test_df.shape)


(4917, 4)
(491, 4)


In [3]:
def clean_string(content):
#     cc = OpenCC('t2s')
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
#     content = cc.convert(content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < 510:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = 510
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li
            
    
    

In [4]:
names =  train_df['name'].tolist()
contents = np.array(train_df['full_content'].tolist())
ckip_names = train_df['ckip_names'].tolist()
x = []
binary_y = []
names_y  = []
content_all_names = []
start_pos_labels = []
end_pos_labels = []
for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])

    if(content=='nan'):
        continue

    
    name_li  = ast.literal_eval(names[i])
    if(len(name_li) == 0 ):
#         content切句 
#         split_content = cut_sent(content)
#         chunks = combine_sentence(split_content)
        
#         for chunk in chunks:
#             start_pos_label = np.zeros(512)
#             end_pos_label = np.zeros(512)
#             binary_y.append(0)
#             start_pos_label[0] = 1
#             end_pos_label[0] = 1
#             x.append(chunk)
#             start_pos_labels.append(start_pos_label)
#             end_pos_labels.append(end_pos_label)
#             content_all_names.append(content_ckip_names)
        binary_y.append(0)
        x.append(content)
        
    else:
        binary_y.append(1)
        x.append(content)
        
#         for name in name_li:
        
#             name = name.replace('\n','').replace('\t','').replace(' ','')
#             #  content切句 
            
#             _pos = 0
#             while True:
#                 start_pos = content.find(name,_pos)
#                 if(start_pos == -1):
#                     break
#                 start_pos += 1
#                 _pos = start_pos
#                 end_pos = start_pos + len(name)

#                 if(start_pos < 512 and end_pos < 512 ):
#                     binary_y.append(1)
#                     x.append(content)

#                     start_pos_label = np.zeros(512)
#                     end_pos_label = np.zeros(512)
#                     start_pos_label[start_pos] = 1
#                     end_pos_label[end_pos] = 1

#                     start_pos_labels.append(start_pos_label)
#                     end_pos_labels.append(end_pos_label)
#                     content_all_names.append(content_ckip_names)
        

        
        
train_x = np.array(x)
train_binary_y = np.array(binary_y)
train_start_pos_labels = np.array(start_pos_labels)
train_end_pos_labels = np.array(end_pos_labels)
train_ckip_names = np.array(content_all_names)


print(train_x.shape)
print(train_binary_y.shape)
print(train_start_pos_labels.shape)
print(train_end_pos_labels.shape)
print(train_ckip_names.shape)

(4917,)
(4917,)
(0,)
(0,)
(0,)


In [5]:
model_path  = './chinese_wwm_pytorch/'

tokenizer = BertTokenizer.from_pretrained(model_path)

train_input_dict = tokenizer.batch_encode_plus(train_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [6]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y

    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return inputid , tokentype , attentionmask, y
    
    def __len__(self):
        return len(self.input_ids)
    

In [7]:

class BertBinrayClassifier(nn.Module):
    def __init__(self):
        super(BertBinrayClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(768, 2),
        )


#             
    def forward(self, x):
        x = self.classifier(x)

        return x 

In [8]:
BATCH_SIZE = 8
trainset = TrainDataset(train_input_dict, 
                        train_binary_y)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print("device:", device)

model_path  = './chinese_wwm_pytorch/'
config = BertConfig.from_pretrained(model_path + 'config.json',output_hidden_states=True)
model = BertModel.from_pretrained(model_path,config=config)
model = model.to(device)
model.eval()

binary_model = BertBinrayClassifier()
binary_model = binary_model.to(device)
binary_model = binary_model.double()
binary_model.train()


weight = torch.tensor([0.1,1]).double().to(device)

loss_fn = nn.CrossEntropyLoss(weight = weight)




EPOCHS = 10
optimizer = torch.optim.Adam(binary_model.parameters(), lr=3e-5)




for epoch in range(0,EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors , binary_y  = [t.to(device) for t in data]
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
  

        bert_all_768 = bert_outputs[1]
        optimizer.zero_grad()
        
        total += bert_all_768.size()[0]
        
        
        logits = binary_model(bert_all_768.double())
        loss = loss_fn(logits , binary_y)
        loss.backward()
        optimizer.step()
        
        
        logits = torch.argmax(logits, dim=-1)
        correct += (logits == binary_y).sum().item()
        running_loss += loss.item()
        
    torch.save(binary_model.state_dict(),'./TB_multispan/Bert_binary_alldataset_' + str(epoch) + '.pkl')
    print(epoch)
    print('train acc:' , correct/total)
    print(running_loss)


            



device: cuda:0
0
train acc: 0.9103111653447223
332.3948754274651
1
train acc: 0.924954240390482
259.30843767182523
2
train acc: 0.9212934716290421
220.9661099026813
3
train acc: 0.9192597112060199
196.8287749353128
4
train acc: 0.9204799674598332
179.9746544690136


KeyboardInterrupt: 